In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
pd.set_option('display.max_columns', None)

# Gathering

### 1- Gathering the twitter archive of WeRateDogs account

In [2]:
twitter_archive = pd.read_csv("Input/twitter-archive-enhanced (2).csv")
print (twitter_archive.shape)
twitter_archive.head(2)

(2356, 17)


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


### 2- Gathering the image predictions

In [3]:
# By using the requests library, we can download the tweet image predictions
import requests

URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
image = requests.get(URL, allow_redirects=True)

open('image_predictions.tsv', 'wb').write(image.content)

335079

In [4]:
images_prediction = pd.read_csv('image_predictions.tsv', sep = '\t')
print (images_prediction.shape)
images_prediction.head(2)

(2075, 12)


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


### 3- Gathering the tweet ID, retweet count, and favorite count from json file

In [5]:
import json

selected = []
with open('Input/tweet-json.txt', 'r') as json_file:
    for line in json_file:
        json_data = json.loads(line)
        selected.append({'tweet_id': json_data['id'],
                             'favorite_count': json_data['favorite_count'],
                             'retweet_count': json_data['retweet_count']})
tweet_detail = pd.DataFrame(selected, columns=['tweet_id','favorite_count','retweet_count'])
print (tweet_detail.shape)
tweet_detail.head(2)

(2354, 3)


,tweet_id,favorite_count,retweet_count
0,892420643555336193,39467,8853
1,892177421306343426,33819,6514


# Assess

##### Quality

**twitter_archive dataset** :

* Completeness:

> The column in twitter_archive dataset, which have missing data, are in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp, and expanded_urls

* Validity:

> dog names: some dogs have 'None' as a name, or 'a', or 'an.'

> There are retweeted tweets, and we do not want it.

> Clean the content of source column


* Accuracy:

> tweet_id column is stored as float. it sholud be str

> timestamp is stored as an object

> retweeted_status_timestamp is stored as an object

* Consistency:


> Standardize dog ratings


##### Tidiness:

> Dog stage is in 4 columns (doggo, floofer, pupper, puppo). It should be one column.

> All the three dataset should be in one dateset


**images_prediction dataset** :

* Completeness:

> Missing values from images dataset (2075 rows instead of 2356).

* Consistency:

> The capitalization are not consistent in columns p1, p2, and p3 


> Underscore for multi-word dog breeds in p1, p2 and p3 columns

**Tidiness**
>The dog breed prediction are in three columns. it should be one column

>The prediction confidence are in three columns. it should be one column

**tweet_detail**

* Validity:

> Rename id to tweet_id in order to do the merging

# twitter_archive

In [6]:
# Many missing value
# tweet_id is float
# time is object
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [7]:
# We have no duplicstes ID
sum(twitter_archive['tweet_id'].duplicated())

0

In [8]:
# None and a, some other names are not Valid

twitter_archive['name'].value_counts()

None            745
a                55
Charlie          12
Cooper           11
Lucy             11
               ... 
Perry             1
Monkey            1
unacceptable      1
Tupawc            1
Blanket           1
Name: name, Length: 957, dtype: int64

In [9]:
# rating_denominator should be 10

twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [10]:
# Orignal tweets (No retweet)
twitter_archive[twitter_archive['retweeted_status_id'].isnull()].shape

(2175, 17)

# images_prediction

In [11]:
images_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [12]:
# The capitalization are not consistent in columns p1, p2, and p3 
# Underscore for multi-word dog breeds in p1, p2 and p3 columns.

images_prediction.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [13]:
images_prediction['p1'].value_counts()

golden_retriever            150
Labrador_retriever          100
Pembroke                     89
Chihuahua                    83
pug                          57
                           ... 
cheetah                       1
electric_fan                  1
tricycle                      1
sulphur-crested_cockatoo      1
traffic_light                 1
Name: p1, Length: 378, dtype: int64

In [14]:
from IPython.display import Image 
Image(url = 'https://pbs.twimg.com/media/CWPHMqKVAAAE78E.jpg')

# tweet_detail

In [15]:
# no missing values
tweet_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int64
 1   favorite_count  2354 non-null   int64
 2   retweet_count   2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


In [16]:
tweet_detail.describe()

,tweet_id,favorite_count,retweet_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,8080.968564,3164.797366
std,6.852812e+16,11814.771334,5284.770364
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,1415.000000,624.500000
50%,7.194596e+17,3603.500000,1473.500000
75%,7.993058e+17,10122.250000,3652.000000
max,8.924206e+17,132810.000000,79515.000000


# Clean

In [17]:
# We have to make a copy of the datasets before we do the cleaning
twitter_archive_clean = twitter_archive.copy()
images_prediction_clean = images_prediction.copy()
tweet_detail_clean = tweet_detail.copy()

#### Define

1. Change tweet_id from an integer to a string
2. Merge twitter_archive, images_prediction, and tweet_detail_clean datastes together
3. Create one column for the dog stage: doggo, floofer, pupper, puppo 
4. Delete retweets
5. Drop the no needed columns such as, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, and retweeted_status_timestamp
6. Change the timestamp from object to datetime format
7. Fixing names issue
8. The capitalization are not consistent in columns p1, p2, and p3 
9. Underscore for multi-word dog breeds in p1, p2 and p3 columns
10. Clean the content of source column
11. Standardize dog ratings
12. The dog breed prediction are in three columns. it should be one column
13. The prediction confidence are in three columns. it should be one column

### 1- Define 

 > Change tweet_id from an integer to a string

### 1- Code 

In [18]:
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)
images_prediction_clean['tweet_id'] = images_prediction_clean['tweet_id'].astype(str)
tweet_detail_clean['tweet_id'] = tweet_detail_clean['tweet_id'].astype(str)

### 1- Test

In [19]:
print (twitter_archive_clean['tweet_id'].dtype)
print (images_prediction_clean['tweet_id'].dtype)
print (tweet_detail_clean['tweet_id'].dtype)

object
object
object


### 2- Define

> Merge twitter_archive, images_prediction, and tweet_detail_clean datastes together

### 2- Code

In [20]:
all_details = pd.merge(twitter_archive_clean,
                                 images_prediction_clean, on='tweet_id', how='inner')

In [21]:
all_details = pd.merge(all_details,
                                 tweet_detail_clean, on='tweet_id', how='inner')

### 2- Test

In [22]:
all_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2072
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2073 non-null   object 
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   2073 non-null   object 
 4   source                      2073 non-null   object 
 5   text                        2073 non-null   object 
 6   retweeted_status_id         79 non-null     float64
 7   retweeted_status_user_id    79 non-null     float64
 8   retweeted_status_timestamp  79 non-null     object 
 9   expanded_urls               2073 non-null   object 
 10  rating_numerator            2073 non-null   int64  
 11  rating_denominator          2073 non-null   int64  
 12  name                        2073 non-null   object 
 13  doggo                       2073 

### 3- Define 

> Create one column for the dog stage: doggo, floofer, pupper, puppo 

### 3- Code 

In [23]:
all_details['dog_stage'] = all_details['text'].str.lower().map(lambda x: 
            np.NaN if "both" in x 
            else "puppo" if "puppo" in x 
            else "doggo" if "doggo" in x 
            else "pupper" if "pupper" in x 
            else "floofer" if "floofer" in x 
            else np.NaN)

### 3- Test

In [24]:
all_details.dog_stage.value_counts()

pupper     228
doggo       74
puppo       30
floofer      7
Name: dog_stage, dtype: int64

### 4- Define
> Delete retweets

### 4- Code 

In [25]:
# We are going to keep the retweeted_status_id is null
all_details= all_details[all_details['retweeted_status_id'].isnull()]
all_details.shape

(1994, 31)

### 4- Test

In [26]:
# Checking thta we do not have any retweeted_status_id exist
all_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2072
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    1994 non-null   object 
 1   in_reply_to_status_id       23 non-null     float64
 2   in_reply_to_user_id         23 non-null     float64
 3   timestamp                   1994 non-null   object 
 4   source                      1994 non-null   object 
 5   text                        1994 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               1994 non-null   object 
 10  rating_numerator            1994 non-null   int64  
 11  rating_denominator          1994 non-null   int64  
 12  name                        1994 non-null   object 
 13  doggo                       1994 

### 5- Define 
> Drop the no needed columns

### 5- Code

In [27]:
# Remove the columns that we do not need
all_details = all_details.drop(['retweeted_status_id', 
                                'retweeted_status_user_id',
                                'retweeted_status_timestamp',
                                'in_reply_to_status_id', 
                                'in_reply_to_user_id',
                                'img_num',
                               'doggo','floofer','pupper','puppo'], axis=1)

### 5- Test

In [28]:
all_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2072
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            1994 non-null   object 
 1   timestamp           1994 non-null   object 
 2   source              1994 non-null   object 
 3   text                1994 non-null   object 
 4   expanded_urls       1994 non-null   object 
 5   rating_numerator    1994 non-null   int64  
 6   rating_denominator  1994 non-null   int64  
 7   name                1994 non-null   object 
 8   jpg_url             1994 non-null   object 
 9   p1                  1994 non-null   object 
 10  p1_conf             1994 non-null   float64
 11  p1_dog              1994 non-null   bool   
 12  p2                  1994 non-null   object 
 13  p2_conf             1994 non-null   float64
 14  p2_dog              1994 non-null   bool   
 15  p3                  1994 non-null   object 
 16  p3_con

### 6- Define
> Change the timestamp from object to datetime format

### 6- Code

In [29]:
all_details['timestamp'] = all_details['timestamp'].str.slice(start=0, stop=-6)

In [30]:
all_details['timestamp'] = pd.to_datetime(all_details['timestamp'], format = "%Y-%m-%d %H:%M:%S")

### 6- Test

In [31]:
all_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2072
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            1994 non-null   object        
 1   timestamp           1994 non-null   datetime64[ns]
 2   source              1994 non-null   object        
 3   text                1994 non-null   object        
 4   expanded_urls       1994 non-null   object        
 5   rating_numerator    1994 non-null   int64         
 6   rating_denominator  1994 non-null   int64         
 7   name                1994 non-null   object        
 8   jpg_url             1994 non-null   object        
 9   p1                  1994 non-null   object        
 10  p1_conf             1994 non-null   float64       
 11  p1_dog              1994 non-null   bool          
 12  p2                  1994 non-null   object        
 13  p2_conf             1994 non-null   float64     

### 7- Define 
> Fixing names issue

### 7- Code

In [32]:
all_details['name'] = all_details['name'].str.replace('^[a-z]+', 'None')

### 7- Test

In [33]:
all_details['name'].value_counts()

None         644
Charlie       11
Oliver        10
Lucy          10
Cooper        10
            ... 
Cheryl         1
Tycho          1
Alexander      1
Arlo           1
Blanket        1
Name: name, Length: 914, dtype: int64

### 8- Define
> The capitalization are not consistent in columns p1, p2, and p3 

### 8- Code

In [34]:
all_details['p1'] = all_details['p1'].str.lower()
all_details['p2'] = all_details['p2'].str.lower()
all_details['p3'] = all_details['p3'].str.lower()

### 8- Test

In [35]:
# all should be lower case
all_details[['p1','p2','p3']]

,p1,p2,p3
0,orange,bagel,banana
1,chihuahua,pekinese,papillon
2,chihuahua,malamute,kelpie
3,paper_towel,labrador_retriever,spatula
4,basset,english_springer,german_short-haired_pointer
...,...,...,...
2068,miniature_pinscher,rottweiler,doberman
2069,rhodesian_ridgeback,redbone,miniature_pinscher
2070,german_shepherd,malinois,bloodhound
2071,redbone,miniature_pinscher,rhodesian_ridgeback


### 9- Define
> Underscore for multi-word dog breeds in p1, p2 and p3 columns

### 9- Code

In [36]:
all_details['p1'] = all_details['p1'].str.replace('_', ' ')
all_details['p2'] = all_details['p2'].str.replace('_', ' ')
all_details['p3'] = all_details['p3'].str.replace('_', ' ')

### 9- Test

In [37]:
# Remove the underscore
all_details[['p1','p2','p3']]

,p1,p2,p3
0,orange,bagel,banana
1,chihuahua,pekinese,papillon
2,chihuahua,malamute,kelpie
3,paper towel,labrador retriever,spatula
4,basset,english springer,german short-haired pointer
...,...,...,...
2068,miniature pinscher,rottweiler,doberman
2069,rhodesian ridgeback,redbone,miniature pinscher
2070,german shepherd,malinois,bloodhound
2071,redbone,miniature pinscher,rhodesian ridgeback


#### 10- Define 
> Clean the content of source column

### 10- Code

In [38]:
all_details['source'] = all_details['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

### 10- Test

In [39]:
all_details['source'].value_counts()

Twitter for iPhone    1955
Twitter Web Client      28
TweetDeck               11
Name: source, dtype: int64

### 11- Define
> Standardize dog ratings

### 11- Code

In [40]:
all_details['rating_numerator'] = all_details['rating_numerator'].astype(float)
all_details['rating_numerator'].dtype

dtype('float64')

In [41]:
ratings_decimals_text = []
ratings_decimals_index = []
ratings_decimals = []

for i, text in all_details['text'].iteritems():
    if bool(re.search('\d+\.\d+\/\d+', text)):
        ratings_decimals_text.append(text)
        ratings_decimals_index.append(i)
        ratings_decimals.append(re.search('\d+\.\d+', text).group())

# Print the ratings which have decimals        
ratings_decimals_text

['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
 "This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",
 "This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq",
 'Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD']

In [42]:
all_details.loc[ratings_decimals_index[0],'rating_numerator'] = float(ratings_decimals[0])
all_details.loc[ratings_decimals_index[1],'rating_numerator'] = float(ratings_decimals[1])
all_details.loc[ratings_decimals_index[2],'rating_numerator'] = float(ratings_decimals[2])
all_details.loc[ratings_decimals_index[3],'rating_numerator'] = float(ratings_decimals[3])

### 11- Test

In [43]:
# The rating of tweet_id = 883482846933004288 was 5/10. right now after it is corrected 13.5/10
all_details.loc[40]

tweet_id                                             883482846933004288
timestamp                                           2017-07-08 00:28:19
source                                               Twitter for iPhone
text                  This is Bella. She hopes her smile made you sm...
expanded_urls         https://twitter.com/dog_rates/status/883482846...
rating_numerator                                                   13.5
rating_denominator                                                   10
name                                                              Bella
jpg_url                 https://pbs.twimg.com/media/DELC9dZXUAADqUk.jpg
p1                                                     golden retriever
p1_conf                                                        0.943082
p1_dog                                                             True
p2                                                   labrador retriever
p2_conf                                                        0

### 12- Define

>The dog breed prediction are in three columns. it should be one column

### 12- Code

In [44]:
def breed(row):
    if row['p1_dog']:
        return(row['p1'])
    elif row['p2_dog']:
        return(row['p2'])
    elif row['p3_dog']:
        return(row['p3'])
    else:
        return(np.NaN)

In [45]:
all_details['breed'] = all_details.apply (lambda row: breed (row),axis=1)

### 12- Test

In [46]:
all_details[['breed']]

,breed
0,NaN
1,chihuahua
2,chihuahua
3,labrador retriever
4,basset
...,...
2068,miniature pinscher
2069,rhodesian ridgeback
2070,german shepherd
2071,redbone


### 13- Define

>The prediction confidence are in three columns. it should be one column

### 13- Code

In [47]:
def confidence(row):
    if row['p1_dog']:
        return(row['p1_conf'])
    elif row['p2_dog']:
        return(row['p2_conf'])
    elif row['p3_dog']:
        return(row['p3_conf'])
    else:
        return(np.NaN)

In [48]:
all_details['confidence'] = all_details.apply (lambda row: confidence(row),axis=1)

### 13- Test

In [49]:
all_details[['confidence']]

,confidence
0,NaN
1,0.323581
2,0.716012
3,0.168086
4,0.555712
...,...
2068,0.560311
2069,0.408143
2070,0.596461
2071,0.506826


In [50]:
# Store the data 
all_details.to_csv('Output/twitter_archive_master.csv', index=False)

## Analyzing and visualizing the data will be in act_report file